In [6]:
# 버킷 폴더 생성
OUTPUT_BUCKET_FOLDER = "gs://capstone-01/output/"
DATA_BUCKET_FOLDER = "gs://capstone-01/data/"

In [7]:
# 패키지 불러오기
from pyspark.sql.types import *
import pyspark.sql.functions as F

## loading data

In [10]:
# 사용자 정의 함수 udf
# timestamp를 넣으면 일자(숫자) 반환
# 예를 들어, events.csv의 timestamp를 이 함수에 넣으면, ad를 클릭한 날이 기준일로부터 몇 일인지 반환. 최소 0일 ~ 최대 12일.
truncate_day_from_timestamp_udf = F.udf(lambda ts: int(ts / 1000 / 60 / 60 / 24), IntegerType())

In [14]:
# events.csv 가져오기
# ad를 클릭한 날(timestamp 정보)이 첫날로부터 몇 일 경과했는지 기록한 day_event 컬럼 추가
# PySpark에서는 파일을 가져올 때 컬럼 type이랑 name 지정해서 가져와야 함
# 파일의 schema를 먼저 설정하고, spark.read.schema로 가져옴

events_schema = StructType(
                    [StructField("display_id", IntegerType(), True),
                    StructField("uuid_event", StringType(), True),                    
                    StructField("document_id_event", IntegerType(), True),
                    StructField("timestamp_event", IntegerType(), True),
                    StructField("platform_event", IntegerType(), True),
                    StructField("geo_location_event", StringType(), True)]
                    )

events_df = spark.read.schema(events_schema).options(header='true', inferschema='false', nullValue='\\N') \
                .csv(DATA_BUCKET_FOLDER + "events.csv") \
                .withColumn('day_event', truncate_day_from_timestamp_udf('timestamp_event')) \
                .alias('events')

In [ ]:
# promoted_content.csv 가져오기

promoted_content_schema = StructType(
                    [StructField("ad_id", IntegerType(), True),
                    StructField("document_id_promo", IntegerType(), True),                    
                    StructField("campaign_id", IntegerType(), True),
                    StructField("advertiser_id", IntegerType(), True)]
                    )

promoted_content_df = spark.read.schema(promoted_content_schema).options(header='true', inferschema='false', nullValue='\\N') \
                .csv(DATA_BUCKET_FOLDER+"promoted_content.csv") \
                .alias('promoted_content')

In [ ]:
# clicks_train.csv 가져오기

clicks_train_schema = StructType(
                    [StructField("display_id", IntegerType(), True),
                    StructField("ad_id", IntegerType(), True),                    
                    StructField("clicked", IntegerType(), True)]
                    )

clicks_train_df = spark.read.schema(clicks_train_schema).options(header='true', inferschema='false', nullValue='\\N') \
                .csv(DATA_BUCKET_FOLDER+"clicks_train.csv") \
                .alias('clicks_train')

In [ ]:
# clicks_train_joined.csv 생성하기
# clicks_train, promoted_content를 join (key='ad_id')
# clicks_train, promoted_content를 events와 join (key='display_id)')
# .createOrReplaceTempView : View를 생성하는 함수

clicks_train_joined_df = clicks_train_df \
                         .join(promoted_content_df, on='ad_id', how='left') \
                         .join(events_df, on='display_id', how='left')                         
clicks_train_joined_df.createOrReplaceTempView('clicks_train_joined')

In [ ]:
# clicks_train_joined에서 display_id, day_event만 select하고 unique한 것만 추출
# validation set을 만들기 위해 display_id 샘플링
# 샘플링 기준 : day_event(ad를 클릭한 날이 첫날로부터 몇 일 경과했는지)가 0~10이면 0.2 추출, 11~12면 1(전부) 추출
# 이 validation set은 clicks_test를 샘플링한 방법과 동일하다
# 즉, clicks_test도 day_event가 0~10인 데이터에서 0.2의 비율로, 11~12인 데이터는 전부 추출해서 만들어졌다

validation_display_ids_df = clicks_train_joined_df.select('display_id','day_event').distinct() \
                                .sampleBy("day_event", fractions={0: 0.2, 1: 0.2, 2: 0.2, 3: 0.2, 4: 0.2, \
                                                                5: 0.2, 6: 0.2, 7: 0.2, 8: 0.2, 9: 0.2, 10: 0.2, \
                                                               11: 1.0, 12: 1.0}, seed=0)   
validation_display_ids_df.createOrReplaceTempView("validation_display_ids")

In [ ]:
# PySpark에서 sql문을 써서 validation_set_df 생성
# clicks_train_joined에서 8개 컬럼 select
# validation set에 해당하는 display_id만 뽑아 validation_set_df에 저장

validation_set_df = spark.sql('''SELECT display_id, ad_id, uuid_event, day_event, timestamp_event,
                                        document_id_promo, platform_event, geo_location_event FROM clicks_train_joined t 
             WHERE EXISTS (SELECT display_id FROM validation_display_ids 
                           WHERE display_id = t.display_id)''')

In [ ]:
# 생성한 validation set을 google storage에 저장

validation_set_gcs_output = "validation_set.parquet"
validation_set_df.write.parquet(OUTPUT_BUCKET_FOLDER+validation_set_gcs_output, mode='overwrite')

In [ ]:
# PySpark에서 take는 첫 n 개 원소를 뽑아낸 배열을 생성하는 함수. 파이썬의 head 함수와 유사.

validation_set_df.take(5)